In [55]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU
from keras.callbacks import ModelCheckpoint
from random import randint
from keras.models import model_from_yaml

In [56]:
with open("hazes.txt") as corpus_file:
    corpus = corpus_file.read()
    corpus = corpus.lower()
print("Loaded a corpus of {0} characters".format(len(corpus)))

Loaded a corpus of 41701 characters


In [57]:
used_chars = sorted(list(set(corpus)))
print(used_chars)

['\n', ' ', '"', "'", '(', ')', ',', '.', '1', '2', ':', ';', '?', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [58]:
# Get a unique identifier for each char in the corpus, then make some dicts to ease encoding and decoding
corpus_length = len(corpus)
chars = sorted(list(set(corpus)))
sentence_length = 20
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}

print(chars)
print("Our corpus contains {0} unique characters.".format(num_chars))
print(encoding)

['\n', ' ', '"', "'", '(', ')', ',', '.', '1', '2', ':', ';', '?', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Our corpus contains 39 unique characters.
{'\n': 0, ' ': 1, '"': 2, "'": 3, '(': 4, ')': 5, ',': 6, '.': 7, '1': 8, '2': 9, ':': 10, ';': 11, '?': 12, '`': 13, 'a': 14, 'b': 15, 'c': 16, 'd': 17, 'e': 18, 'f': 19, 'g': 20, 'h': 21, 'i': 22, 'j': 23, 'k': 24, 'l': 25, 'm': 26, 'n': 27, 'o': 28, 'p': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38}


In [59]:
with open('model.yaml') as model_file:
    architecture = model_file.read()
model = model_from_yaml(architecture)

In [60]:
model.load_weights('weights-20.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               86016     
_________________________________________________________________
dense (Dense)                (None, 39)                5031      
_________________________________________________________________
activation (Activation)      (None, 39)                0         
Total params: 91,047
Trainable params: 91,047
Non-trainable params: 0
_________________________________________________________________


In [61]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [62]:
def generate(seed_pattern):
        X = np.zeros((1, sentence_length, num_chars), dtype=np.bool)
        #print(X.shape)
        for i, character in enumerate(seed_pattern):
            X[0, i, encoding[character]] = 1
        
        generated_text = ""
        for i in range(500):
            # even de temperatuur toevoegen.
            prediction = sample(model.predict(X, verbose=0)[0],0.3)
            generated_text += decoding[prediction]

            activations = np.zeros((1, 1, num_chars), dtype=np.bool)
            activations[0, 0, prediction] = 1
            X = np.concatenate((X[:, 1:, :], activations), axis=1)

        return generated_text

def make_seed(seed_phrase=""):
        if seed_phrase:
            phrase_length = len(seed_phrase)
            pattern = ""
            for i in range (0, sentence_length):
                pattern += seed_phrase[i % phrase_length]
        else:
            seed = randint(0, corpus_length - sentence_length)
            pattern = corpus[seed:seed + sentence_length]

        return pattern

In [63]:
seed = make_seed('ik houd zo van kaas')
print(seed)
txt =  generate(seed)
print(txt)

ik houd zo van kaasi
k voor mij alleen
ik wil vendaan mijn leven toch geen truik
het is de laatste beder
dan toch ik heb ik het tou streen voor me nooit
en de kruin dan het geleeft maar dat mijnen kenden beren
maar dat ik het nooit met blijen
maar dan het is de krekken me staan van je huur
dan den zegen wag ik me heen
det ik heb voor mij alleen staat
maar jij nu staan zij wanden de kroeden
zijn kom niet maar dat men
en kenken wat ik van je heen
ik heb ik het nooit gevoeld
nee, zo had ik het niet verdacht
dat ik je n
